In [1]:
import torch.cuda
from importlib import reload
import networks
reload(networks)
from networks.default_cnn import default_cnn
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import datetime

torch.manual_seed(42)


in_size = 128
out_size = 2

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
network = default_cnn(in_size, out_size)

In [3]:
transform = transforms.Compose([
    transforms.Resize((in_size, in_size)),
    transforms.ToTensor()
])
print(device)

cuda


In [4]:
# Create a dataset from image folder
dataset = ImageFolder(root='/home/gatemrou/uds/Eye_Tracking/data/cropped/heatmaps_sorted', transform=transform)

# Define the proportions for train, val, and test sets
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Calculate the sizes of each subset based on the ratios
train_size = int(train_ratio * len(dataset))
val_size = int(val_ratio * len(dataset))
test_size = len(dataset) - train_size - val_size

# Randomly split the dataset into train, val, and test subsets
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders for each subset
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [5]:
learning_rate = 0.02

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(network.parameters(), lr=learning_rate)

In [6]:
# training
network = network.to(device)
start_time = datetime.datetime.now().strftime("%m.%d_%H:%M:%S")
print(f'{start_time} --- Training started')
num_epochs = 100
best_valid_accuracy = 0
network.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = network.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    # loss and evaluation after each epoch
    network.eval()
    valid_accuracy = 0.0
    valid_samples = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = network(inputs)
            _, predicted = torch.max(outputs, 1)
            valid_accuracy += (predicted == labels).sum().item()
            valid_samples += labels.size(0)
    
    valid_accuracy /= valid_samples
    avg_loss = running_loss / len(train_loader)
    print('{} --- Epoch [{}/{}], Loss: {:.4f}, Validation Accuracy: {:.2f}%'.format(
        datetime.datetime.now().strftime("%m.%d_%H:%M:%S"), epoch+1, num_epochs, avg_loss, valid_accuracy * 100))

03.03_21:26:39 --- Training started
03.03_21:26:41 --- Epoch [1/100], Loss: 0.5684, Validation Accuracy: 68.63%
03.03_21:26:42 --- Epoch [2/100], Loss: 0.5102, Validation Accuracy: 68.63%
03.03_21:26:43 --- Epoch [3/100], Loss: 0.5040, Validation Accuracy: 68.63%
03.03_21:26:45 --- Epoch [4/100], Loss: 0.4972, Validation Accuracy: 68.63%
03.03_21:26:46 --- Epoch [5/100], Loss: 0.4672, Validation Accuracy: 68.63%
03.03_21:26:47 --- Epoch [6/100], Loss: 0.4537, Validation Accuracy: 68.63%
03.03_21:26:48 --- Epoch [7/100], Loss: 0.4936, Validation Accuracy: 31.37%
03.03_21:26:50 --- Epoch [8/100], Loss: 0.5694, Validation Accuracy: 68.63%
03.03_21:26:51 --- Epoch [9/100], Loss: 0.5101, Validation Accuracy: 68.63%
03.03_21:26:52 --- Epoch [10/100], Loss: 0.5556, Validation Accuracy: 68.63%
03.03_21:26:53 --- Epoch [11/100], Loss: 0.5384, Validation Accuracy: 68.63%
03.03_21:26:54 --- Epoch [12/100], Loss: 0.5170, Validation Accuracy: 68.63%
03.03_21:26:56 --- Epoch [13/100], Loss: 0.4802, 

In [7]:
network.eval()
total_samples = 0
correct_predictions = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = network.forward(inputs)

        # Get predictions
        _, predicted = torch.max(outputs, 1)

        # Update counts
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

# Calculate accuracy
accuracy = correct_predictions / total_samples
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 88.46%
